# zawęzanie okna
## nazwa funkcji: limiter(DF,name,main_category,category,goal_usd,duration,country,currency)
#### argumenty (input): 
* DF - data frame do ucięcia
* name - nazwa kampanii
* main category - wiadomo
* category - wiadomo
* goal - wnioskowana kwota, przeliczona na dolary
* duration - czas trwania kampanii
* country - wiadomo
* currency - wiadomo



#### output: DF_limited

#### założenia
* na podstawie wprowadzonych danych funkcja ma za zadanie ograniczyć rozmiar dataframe'u do projektów, które są "podobne" do ocenianego przez apkę 
* TYMCZASOWO - funkcja będzie zawężać pełny data frame, otwarty tutaj, docelowo będzie korzystać z data frame otwartego przez inny kawałek apki
* TYMCZASOWO - koszyki dla wnioskowanej kwoty, czasu trwania kampanii i krajów powyznaczam "jakoś" na podstawie notebooka z analizami, doszlifuję je w dalszym etapie w razie potrzeby

### DF

In [19]:
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt

file_path = (glob.glob(os.path.join(os.path.abspath(''), '**', '*kickstarter_filtered*.tsv'), recursive=True))

DF = pd.read_csv(file_path[0], header = 0, sep='\t')
DF


,id,name,main_category,category,country,launched,deadline,duration,currency,goal_in_defined_currency,pledged_in_defined_currency,goal_in_usd,pledged_in_usd,percentage_of_money_collected,backers,state
0,1860890148,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,United States,2009-04-21,2009-05-31,40,USD,1000.0,625.0,1000.00,625.00,0.625000,30,failed
1,709707365,CRYSTAL ANTLERS UNTITLED MOVIE,Film & Video,Shorts,United States,2009-04-23,2009-07-20,88,USD,80000.0,22.0,80000.00,22.00,0.000275,3,failed
2,1703704063,drawing for dollars,Art,Illustration,United States,2009-04-24,2009-05-03,9,USD,20.0,35.0,20.00,35.00,1.750000,3,successful
3,727286,Offline Wikipedia iPhone app,Technology,Software,United States,2009-04-25,2009-07-14,80,USD,99.0,145.0,99.00,145.00,1.464646,25,successful
4,1622952265,Pantshirts,Fashion,Fashion,United States,2009-04-27,2009-05-26,29,USD,1900.0,387.0,1900.00,387.00,0.203684,10,failed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368233,1315429286,Healthcare Web TV Network,Journalism,Web,United States,2017-12-29,2018-01-18,20,USD,100000.0,0.0,100000.00,0.00,0.000000,0,canceled
368234,801423615,Collaboration with Snoop Dogg,Music,Hip-Hop,Australia,2017-12-29,2018-01-01,3,AUD,500.0,655.0,392.06,513.60,1.310004,6,successful
368235,1057488050,15 Games,Games,Tabletop Games,United States,2017-12-31,2018-01-05,5,USD,1000.0,1.0,1000.00,1.00,0.001000,1,canceled
368236,1854247643,100 Acres of Hell,Comics,Comic Books,Great Britain (UK),2018-01-01,2018-01-15,14,GBP,100.0,57.0,136.40,77.75,0.570015,6,canceled


### Koszyki

In [37]:
#print(DF['goal_in_usd'].describe())

def basket_goal(goal):
    if goal <= 500:
        goal_min = 0
        goal_max = 600
    elif goal <= 1000:
        goal_min = 300
        goal_max = 1200
    elif goal <= 2000:
        goal_min = 800
        goal_max = 2500
    elif goal <= 4000:
        goal_min = 1500
        goal_max = 5000
    elif goal <= 10000:
        goal_min = 4000
        goal_max = 12000
    elif goal <= 20000:
        goal_min = 8000
        goal_max = 25000
    elif goal <= 40000:
        goal_min = 16000
        goal_max = 50000
    elif goal < 100000:
        goal_min = 30000
        goal_max = 120000
    else:
        goal_min = 80000
        goal_max = 20000000
    return[goal_min, goal_max]


In [ ]:
def basket_dur(dur):
    if dur < 30:
        dur_min = 0
        dur_max = 29
    elif dur == 30:
        dur_min = 30
        dur_max = 30
    else:
        dur_min = 31
        dur_max = 100
    return[dur_min, dur_max]

In [120]:
# koszykownik dla kategorii nieskończony, względnie dobrane koszyki tylko dla main category Technology

def basket_cat(mcat,cat):
    if mcat == 'Art':
        if cat in ['Sculpture',
                 'Conceptual Art',
                 'Illustration',
                 'Public Art',
                 'Mixed Media',
                 'Video Art',
                 'Art',
                 'Installations',
                 'Ceramics',
                 'Textiles',
                 'Painting',
                 'Digital Art',
                 'Performance Art']:
            return(['Sculpture',
                 'Conceptual Art',
                 'Illustration',
                 'Public Art',
                 'Mixed Media',
                 'Video Art',
                 'Art',
                 'Installations',
                 'Ceramics',
                 'Textiles',
                 'Painting',
                 'Digital Art',
                 'Performance Art'])
    elif mcat == 'Comics':
        if cat in ['Comic Books',
                 'Events',
                 'Anthologies',
                 'Graphic Novels',
                 'Comics',
                 'Webcomics']:
            return(['Comic Books',
                 'Events',
                 'Anthologies',
                 'Graphic Novels',
                 'Comics',
                 'Webcomics'])
    elif mcat == 'Crafts':
        if cat in ['Weaving',
                 'Letterpress',
                 'Crochet',
                 'Candles',
                 'Taxidermy',
                 #'Crafts',
                 'Knitting',
                 'Embroidery',
                 'Pottery',
                 'Glass',
                 'Quilts',
                 'Printing',
                 'DIY',
                 'Stationery',
                 'Woodworking']:
            return(['Weaving',
                 'Letterpress',
                 'Crochet',
                 'Candles',
                 'Taxidermy',
                 #'Crafts',
                 'Knitting',
                 'Embroidery',
                 'Pottery',
                 'Glass',
                 'Quilts',
                 'Printing',
                 'DIY',
                 'Stationery',
                 'Woodworking'])
        elif cat in ['Crafts']:
            return(['Crafts'])
    elif mcat == 'Dance':
        if cat in ['Dance', 'Spaces', 'Residencies', 'Workshops', 'Performances']:
            return(['Dance', 'Spaces', 'Residencies', 'Workshops', 'Performances'])
    elif mcat == 'Design':
        if cat in ['Graphic Design',
                 'Product Design',
                 'Civic Design',
                 'Interactive Design',
                 'Architecture',
                 'Typography',
                 'Design']:
            return(['Graphic Design',
                 'Product Design',
                 'Civic Design',
                 'Interactive Design',
                 'Architecture',
                 'Typography',
                 'Design'])
    elif mcat == 'Fashion':
        if cat in ['Couture',
                 'Childrenswear',
                 'Footwear',
                 'Fashion',
                 'Accessories',
                 'Jewelry',
                 'Apparel',
                 'Ready-to-wear',
                 'Pet Fashion']:
            return(['Couture',
                 'Childrenswear',
                 'Footwear',
                 'Fashion',
                 'Accessories',
                 'Jewelry',
                 'Apparel',
                 'Ready-to-wear',
                 'Pet Fashion'])
    elif mcat == 'Film & Video':
        if cat in ['Television',
                 'Horror',
                 'Film & Video',
                 'Science Fiction',
                 'Music Videos',
                 'Documentary',
                 'Family',
                 'Animation',
                 'Fantasy',
                 'Experimental',
                 'Movie Theaters',
                 'Comedy',
                 'Webseries',
                 'Festivals',
                 'Romance',
                 'Drama',
                 'Shorts',
                 'Narrative Film',
                 'Thrillers',
                 'Action']:
            return(['Television',
                 'Horror',
                 'Film & Video',
                 'Science Fiction',
                 'Music Videos',
                 'Documentary',
                 'Family',
                 'Animation',
                 'Fantasy',
                 'Experimental',
                 'Movie Theaters',
                 'Comedy',
                 'Webseries',
                 'Festivals',
                 'Romance',
                 'Drama',
                 'Shorts',
                 'Narrative Film',
                 'Thrillers',
                 'Action'])
    elif mcat == 'Food':
        if cat in ['Spaces',
                 'Vegan',
                 'Drinks',
                 'Restaurants',
                 'Food',
                 'Events',
                 'Farms',
                 'Cookbooks',
                 "Farmer's Markets",
                 'Food Trucks',
                 'Community Gardens',
                 'Small Batch',
                 'Bacon']:
            return(['Spaces',
                 'Vegan',
                 'Drinks',
                 'Restaurants',
                 'Food',
                 'Events',
                 'Farms',
                 'Cookbooks',
                 "Farmer's Markets",
                 'Food Trucks',
                 'Community Gardens',
                 'Small Batch',
                 'Bacon'])
    elif mcat == 'Games':
        if cat in ['Playing Cards',
                 'Puzzles',
                 'Games',
                 'Mobile Games',
                 'Gaming Hardware',
                 'Video Games',
                 'Tabletop Games',
                 'Live Games']:
            return(['Playing Cards',
                 'Puzzles',
                 'Games',
                 'Mobile Games',
                 'Gaming Hardware',
                 'Video Games',
                 'Tabletop Games',
                 'Live Games'])
    elif mcat == 'Journalism':
        if cat in ['Print', 'Video', 'Journalism', 'Web', 'Photo', 'Audio']:
            return(['Print', 'Video', 'Journalism', 'Web', 'Photo', 'Audio'])
    elif mcat == 'Music':
        if cat in ['Classical Music',
                 'Faith',
                 'Chiptune',
                 'Blues',
                 'Indie Rock',
                 'Pop',
                 'Kids',
                 'Hip-Hop',
                 'World Music',
                 'Jazz',
                 'Latin',
                 'Metal',
                 'Comedy',
                 'Country & Folk',
                 'R&B',
                 'Rock',
                 'Music',
                 'Punk',
                 'Electronic Music']:
            return(['Classical Music',
                 'Faith',
                 'Chiptune',
                 'Blues',
                 'Indie Rock',
                 'Pop',
                 'Kids',
                 'Hip-Hop',
                 'World Music',
                 'Jazz',
                 'Latin',
                 'Metal',
                 'Comedy',
                 'Country & Folk',
                 'R&B',
                 'Rock',
                 'Music',
                 'Punk',
                 'Electronic Music'])
    elif mcat == 'Photography':
        if cat in ['Photobooks',
                 'Animals',
                 'Places',
                 'Nature',
                 'People',
                 'Fine Art',
                 'Photography']:
            return(['Photobooks',
                 'Animals',
                 'Places',
                 'Nature',
                 'People',
                 'Fine Art',
                 'Photography'])
    elif mcat == 'Publishing':
        if cat in ['Radio & Podcasts',
                 'Art Books',
                 'Literary Journals',
                 'Poetry',
                 'Calendars',
                 'Comedy',
                 'Letterpress',
                 'Literary Spaces',
                 "Children's Books",
                 'Zines',
                 'Academic',
                 'Publishing',
                 'Anthologies',
                 'Periodicals',
                 'Nonfiction',
                 'Fiction',
                 'Translations',
                 'Young Adult']:
            return(['Radio & Podcasts',
                 'Art Books',
                 'Literary Journals',
                 'Poetry',
                 'Calendars',
                 'Comedy',
                 'Letterpress',
                 'Literary Spaces',
                 "Children's Books",
                 'Zines',
                 'Academic',
                 'Publishing',
                 'Anthologies',
                 'Periodicals',
                 'Nonfiction',
                 'Fiction',
                 'Translations',
                 'Young Adult'])
    elif mcat == 'Technology':
        if cat in [#'Technology',
                 'Wearables',
                 #'Apps',
                 'Space Exploration',
                 'Sound',
                 #'Gadgets',
                 #'Software',
                 '3D Printing',
                 'Robots',
                 'DIY Electronics',
                 #'Hardware',
                 #'Web',
                 'Makerspaces',
                 'Camera Equipment',
                 'Flight',
                 'Fabrication Tools']:
            return([#'Technology',
                 'Wearables',
                 #'Apps',
                 'Space Exploration',
                 'Sound',
                 #'Gadgets',
                 #'Software',
                 '3D Printing',
                 'Robots',
                 'DIY Electronics',
                 #'Hardware',
                 #'Web',
                 'Makerspaces',
                 'Camera Equipment',
                 'Flight',
                 'Fabrication Tools'])
        else:
            return([cat])
    elif mcat == 'Theater':
        if cat in ['Spaces',
                 'Comedy',
                 'Festivals',
                 'Experimental',
                 'Musical',
                 'Theater',
                 'Plays',
                 'Immersive']:
            return(['Spaces',
                 'Comedy',
                 'Festivals',
                 'Experimental',
                 'Musical',
                 'Theater',
                 'Plays',
                 'Immersive'])


#list(set(DF[DF['main_category'] == 'Theater']['category'].to_numpy()))


###  Wymagane argumenty dla funkcji - docelowo importowane z innego notebooka

In [137]:
pr_name = '3 axis video stabilizer for cell phones'
pr_main_category = 'Technology'
pr_category = 'Gadgets'
pr_goal_usd = 7500
pr_duration = 45
pr_country = 'Hong Kong'
pr_currency = 'USD'

### Ograniczanie wielkości ramki
# w rozbudowie :)

In [138]:
def limiter(df,name,mcat,cat,goal,dur,country,curr):

    goal_min = basket_goal(goal)[0]
    goal_max = basket_goal(goal)[1]
    war_goal_min = df['goal_in_usd'] >= goal_min
    war_goal_max = df['goal_in_usd'] <= goal_max
    
    war_mcat = df['main_category'] == mcat
    
    dur_min = basket_dur(dur)[0]
    dur_max = basket_dur(dur)[1]
    war_dur_min = df['duration'] >= dur_min
    war_dur_max = df['duration'] <= dur_max
    
    cat_list = basket_cat(mcat,cat)
    #print(cat_list)
    war_cat_list = df['category'].isin(cat_list)
    #print(war_cat_list)
    
    return(df.loc[war_cat_list][(war_goal_min) & (war_goal_max) & (war_mcat) & (war_dur_max) & (war_dur_min)] )

DF_limited = limiter(DF,pr_name,pr_main_category,pr_category,pr_goal_usd,pr_duration,pr_country,pr_currency)

DF_limited

    

/home/bartosz/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,id,name,main_category,category,country,launched,deadline,duration,currency,goal_in_defined_currency,pledged_in_defined_currency,goal_in_usd,pledged_in_usd,percentage_of_money_collected,backers,state
79379,1227561174,PIXEL: Interactive LED Art,Technology,Gadgets,United States,2013-01-08,2013-03-09,60,USD,8000.0,51536.75,8000.00,51536.75,6.442094,303,successful
142972,1533193928,RECAP USB - Record Calls into Mac and PCs,Technology,Gadgets,United States,2014-05-26,2014-07-25,60,USD,8000.0,2729.00,8000.00,2729.00,0.341125,48,failed
144301,1947354869,SunCache - A Powerful and Flexible Solar Charg...,Technology,Gadgets,United States,2014-06-03,2014-07-18,45,USD,7500.0,24761.00,7500.00,24761.00,3.301467,257,successful
147802,217417568,Inot Ball Lock,Technology,Gadgets,Canada,2014-06-24,2014-08-23,60,CAD,10000.0,21.00,9213.19,19.35,0.002100,3,failed
148293,2032687971,Revolutionary Hands Free Device,Technology,Gadgets,United States,2014-06-26,2014-08-25,60,USD,10000.0,355.00,10000.00,355.00,0.035500,11,failed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363505,1884640869,OXYLURE - Oxygen Powered Fishing Lure - Catch ...,Technology,Gadgets,United States,2017-10-31,2017-12-01,31,USD,10000.0,291.00,10000.00,291.00,0.029100,8,canceled
363628,1191542663,Tayogo Shuffle Mate-Make iPod Shuffle a Waterp...,Technology,Gadgets,United States,2017-11-01,2017-12-05,34,USD,5000.0,236.00,5000.00,236.00,0.047200,9,failed
364925,1126625814,IntelliTouch+: The Smart Screen Protector for ...,Technology,Gadgets,Canada,2017-11-08,2017-12-13,35,CAD,15000.0,8565.00,11750.88,6709.75,0.571000,260,canceled
364931,633816515,Moonracer by Millclock – futuristic steampunk ...,Technology,Gadgets,Norway,2017-11-08,2017-12-10,32,NOK,40000.0,51792.00,4827.24,6250.32,1.294802,23,successful
